In [1]:
import pandas as pd
import numpy as np
#!pip install pandas --upgrade
df = pd.read_csv('crimes.csv')

In [61]:
poverty = pd.read_csv('poverty.csv')
poverty = poverty.drop('COMMUNITY AREA NAME', 1)
poverty.head()

,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [64]:
#  Remove unwanted columns from the data
# Add 'Date' back in when RJ fixes the datetime converter
useful_columns = ['IUCR', 'Location Description', 'Arrest', 'Domestic', 'Community Area', 'Year']
data = df[useful_columns]

In [65]:
# Merge with the poverty dataframe
data.rename(columns={'Community Area': 'Community Area Number'}, inplace=True)
data = pd.merge(data, poverty, how='right', on='Community Area Number')

In [66]:
data.head()

,IUCR,Location Description,Arrest,Domestic,Community Area Number,Year,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,2820,RESIDENCE,False,True,30.0,2012,15.2,30.7,15.8,54.8,33.8,10402,96.0
1,0560,SIDEWALK,True,False,30.0,2012,15.2,30.7,15.8,54.8,33.8,10402,96.0
2,0460,STREET,False,False,30.0,2012,15.2,30.7,15.8,54.8,33.8,10402,96.0
3,1811,SIDEWALK,True,False,30.0,2012,15.2,30.7,15.8,54.8,33.8,10402,96.0
4,0460,STREET,False,False,30.0,2012,15.2,30.7,15.8,54.8,33.8,10402,96.0
